In [48]:
from pymongo import MongoClient
from getpass import getpass
import re
import tqdm
import datetime

In [5]:
# query

host = "localhost"
port = 27017

user = getpass("user mongo")
password = getpass("password mongo")
database_name = getpass("database name")

uri = f"mongodb://{user}:{password}@{host}:{port}?authSource=admin"

In [6]:
client = MongoClient(uri)
db = client[database_name]
collection = db["sentiment-analysis"]

In [52]:
query = {
    "source": "portuguese-tweets-for-sentiment-analysis"
}

print(collection.count_documents(query))

response = collection.find(query)

889312


In [12]:
!python -m spacy download pt_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
  Using cached https://github.com/explosion/spacy-models/releases/download/pt_core_news_sm-3.8.0/pt_core_news_sm-3.8.0-py3-none-any.whl (13.0 MB)
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


DEPRECATION: Loading egg at c:\programdata\miniconda3\lib\site-packages\spirecomm-0.6.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [45]:
import spacy
from num2words import num2words
from unidecode import unidecode
nlp = spacy.load("pt_core_news_sm")
nlp

In [38]:
# code genereted by chatgpt
def numeros_por_extenso_pt(texto: str) -> str:
    def _convert(match):
        token = match.group(0)
        # normaliza separadores decimais pt/en
        if "," in token and "." in token:
            # Ex.: 1.234,56 → remove pontos de milhar e trata vírgula como decimal
            inteiro, frac = token.replace(".", "").split(",", 1)
            inteiro_w = num2words(int(inteiro), lang="pt_BR")
            frac_w = " ".join(num2words(int(d), lang="pt_BR") for d in frac)
            return f"{inteiro_w} vírgula {frac_w}"
        elif "," in token:
            inteiro, frac = token.split(",", 1)
            inteiro_w = num2words(int(inteiro), lang="pt_BR")
            frac_w = " ".join(num2words(int(d), lang="pt_BR") for d in frac)
            return f"{inteiro_w} vírgula {frac_w}"
        elif "." in token:
            # Pode ser decimal estilo EN ou milhar. Se parecer decimal curto, trata como decimal.
            partes = token.split(".")
            if len(partes) == 2 and len(partes[1]) <= 2:
                inteiro, frac = partes
                inteiro_w = num2words(int(inteiro), lang="pt_BR")
                frac_w = " ".join(num2words(int(d), lang="pt_BR") for d in frac)
                return f"{inteiro_w} ponto {frac_w}"
            # senão: milhar -> remove pontos e converte inteiro
            token = token.replace(".", "")
        # inteiro puro
        try:
            return num2words(int(token), lang="pt_BR")
        except ValueError:
            return token  # fallback

    # \d{1,3}(\.\d{3})* para milhar com ponto | \d+ para inteiros | vírgula/ponto para decimais
    padrao = re.compile(r'\b\d{1,3}(?:\.\d{3})*(?:[,\.]\d+)?\b')
    return padrao.sub(_convert, texto)


numeros_por_extenso_pt("13.556,3")

'treze mil, quinhentos e cinquenta e seis vírgula três'

In [53]:
def preprocess(text):
    text = text.lower()
    text = unidecode(text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    doc = nlp(text)    
    tokens_limpos = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    
    return " ".join(tokens_limpos)


for doc in tqdm.tqdm(response):
    text = doc['text']
    collection.update_one(
        {"_id": doc["_id"]},
        {"$set": {"preprocessed_text": preprocess(text),
                  "text_processed_at": datetime.datetime.now(datetime.timezone.utc)}}
    )

889312it [58:30, 253.35it/s]
